In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from keras.utils.np_utils import to_categorical
from keras import models
from keras import layers
from keras.callbacks import EarlyStopping
from collections import Counter

# 1. Load in Data and Train-Validation Split

In [ ]:
df=pd.read_json('../input/train.json')

In [ ]:
df.cuisine = pd.Categorical(df.cuisine)

In [ ]:
df['cuisinecode']=df.cuisine.cat.codes

In [ ]:
test=pd.read_json('../input/test.json')

In [ ]:
df.shape

In [ ]:
test.shape

In [ ]:
df.columns

In [ ]:
test.columns

In [ ]:
df['length']=df['ingredients'].map(len)

In [ ]:
test['length']=test['ingredients'].map(len)

In [ ]:
train, valid = train_test_split(df, 
                                stratify=df['cuisine'], 
                                test_size=.2,
                                random_state=42)

In [ ]:
train=train.copy()

In [ ]:
valid=valid.copy()

# B. Make vocabulary from the ingredients and the words in the ingredients

In [ ]:
train.head()

In [ ]:
cuisine_list=list(train.cuisine.value_counts().index)

In [ ]:
mu=train['length'].mean()

In [ ]:
sigma=train['length'].std()

In [ ]:
train['length']=(train['length']-mu)/sigma
valid['length']=(valid['length']-mu)/sigma
test['length']=(test['length']-mu)/sigma

In [ ]:
train.head()

In [ ]:
def breakout_ingredients(df):
    df['ingredients2']=[" ".join(entry) for entry in df['ingredients']]
    df['ingredients3']=[entry.split(" ") for entry in df['ingredients2']]
    
    def combine(row):
        first=set(row['ingredients'])
        second=set(row['ingredients3'])
        return list(first.union(second))
    
    df['ingredients4']=df.apply(combine, axis=1)
    

In [ ]:
breakout_ingredients(train)
breakout_ingredients(valid)
breakout_ingredients(test)

In [ ]:
# Code courtesy of http://www.davidsbatista.net/blog/2018/02/28/TfidfVectorizer/

def dummy_fun(doc):
    return doc

cv = CountVectorizer(
    analyzer='word',
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None,
    ngram_range=(1,1),
    max_df=.99,
    min_df=1)  

In [ ]:
cv.fit(train.ingredients4)

In [ ]:
vocab_size=len(cv.vocabulary_)

In [ ]:
print("The vocabulary size is {}".format(vocab_size))

In [ ]:
train_vectorized=cv.fit_transform(train['ingredients4'])

In [ ]:
valid_vectorized=cv.transform(valid['ingredients4'])

In [ ]:
test_vectorized=cv.transform(test['ingredients4'])

In [ ]:
one_hot_train_labels=to_categorical(train['cuisinecode'])

In [ ]:
one_hot_valid_labels=to_categorical(valid['cuisinecode'])

In [ ]:
def add_column(source_matrix, df, source_column):
    try:
        source_matrix=source_matrix.todense()
    except:
        print ("Already dense")
    length_matrix=df[source_column].as_matrix()
    length_matrix=length_matrix.reshape((length_matrix.shape[0],1))
    print ("Source shape", source_matrix.shape)
    print ("Column shape", length_matrix.shape)
    result=np.hstack([source_matrix, length_matrix])
    print ("Result shape", result.shape)
    return result
    

In [ ]:
train_vectorized=add_column(train_vectorized, train, 'length')
print ("Train done")
valid_vectorized=add_column(valid_vectorized, valid, 'length')
print ("Valid done")
test_vectorized=add_column(test_vectorized, test, 'length')
print ("Test done")

In [ ]:
one_hot_train_labels.shape

In [ ]:
one_hot_valid_labels.shape

In [ ]:
extra_cols=1

In [ ]:
model=models.Sequential()

In [ ]:
model.add(layers.Dense(6000, 
                       activation='relu', 
                       input_shape=(vocab_size+extra_cols,)))

In [ ]:
model.add(layers.Dropout(0.9)) # refers to nodes in the first hidden layer

In [ ]:
model.add(layers.Dense(1024, 
                       activation='relu', 
                       ))

In [ ]:
model.add(layers.Dropout(0.8)) # refers to nodes in the first hidden layer

In [ ]:
model.add(layers.Dense(20, activation='softmax'))

In [ ]:
model.compile(optimizer='Adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=1, mode='auto')

callbacks_list = [early_stop]


In [ ]:
history=model.fit(train_vectorized,
                 one_hot_train_labels,
                 epochs=20,
                 batch_size=512,
                 validation_data=(valid_vectorized, one_hot_valid_labels),
                  callbacks=callbacks_list)

In [ ]:
plt.figure(1, figsize=(10, 5))

history_dict=history.history
loss_values=history_dict['loss']
val_loss_values = history_dict['val_loss']

epochs = range(1, len(loss_values)+1)

plt.plot(epochs, loss_values, 'bo', label='Training loss')
plt.plot(epochs, val_loss_values, 'b', label = 'Validation loss')

plt.title('Training and validation loss')

plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

In [ ]:
plt.figure(1, figsize=(10, 5))

history_dict=history.history
loss_values=history_dict['acc']
val_loss_values = history_dict['val_acc']

epochs = range(1, len(loss_values)+1)

plt.plot(epochs, loss_values, 'bo', label='Training accuracy')
plt.plot(epochs, val_loss_values, 'b', label = 'Validation accuracy')

plt.title('Training and validation accuracy')

plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

In [ ]:
answer=model.predict_classes(test_vectorized)

In [ ]:
answer.shape

In [ ]:
test['guess']=answer

In [ ]:
test['cuisine']=test['guess'].map(lambda x: df.cuisine.cat.categories[x])

In [ ]:
submission=test[['id','cuisine']]

In [ ]:
submission.to_csv('submission.csv', index=False)

In [ ]:
submission.head()